# Cifar10 classification tricks

In this notebook you will download the cifar10 dataset which contains quite small images (32x32x3) of 10 classes. The data is from the Canadian Institute For Advanced Research. You will plot examples of the images with the class label. Note that because the images are so small it is not always very easy to recognise which of the ten classes is on the image, even as a human. After loading the dataset you will train multiple models and compare the performances of the models on the testset.

**Dataset:**  You work with the Cifar10 dataset. You have 60'000 32x32 pixel color images of 10 classes ("airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck")

**Content:**
* load the original cifar10 data create a train val and test dataset
* visualize samples of cifar10 dataset

* train a random forest on the pixelvalues
* train a cnn from scratch without normalization
* train a cnn from scratch with normalization
* train a cnn from scratch with dropout
* train a cnn from scratch with batchnorm
* train a cnn from scratch with data augmentation

* compare the performances of the models






* 🔧  your task at the end of the notebook: try to beat the models from this notebook


#### Imports

In the next two cells, we load all the required libraries and functions.

In [ ]:
# load required libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
from sklearn.metrics import confusion_matrix

import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
import torch # not needed yet

print(f'Keras_version: {keras.__version__}')# 3.5.0
print(f'torch_version: {torch.__version__}')# 2.5.1+cu121
print(f'keras backend: {keras.backend.backend()}')

# Keras Building blocks
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten , Activation,Dropout,BatchNormalization
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import optimizers


# dataloader and augmentation strategy
from torch.utils.data import TensorDataset, DataLoader
from torchvision import transforms as tr


### Dataloader

In [ ]:
class NumpyDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        return image, label



def plot_history(history):
  # plot the development of the accuracy and loss during training
  plt.figure(figsize=(12,4))
  plt.subplot(1,2,(1))
  plt.plot(history.history['accuracy'],linestyle='-.')
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'valid'], loc='lower right')
  plt.subplot(1,2,(2))
  plt.plot(history.history['loss'],linestyle='-.')
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'valid'], loc='upper right')


### Load and plot the data

In the next cell you will load the Cifar10 dataset, 50'000 images are in the training set and 10'000 are in the test dataset. You will use 10'000 for the train and validation dataset.
You will plot one random example of each label and will see
that the images are really small and finally you can convert the lables into the one hot encoding.


In [ ]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# separate train val and test dataset
X_train=x_train[0:10000]
Y_train=to_categorical(y_train[0:10000],10) # one-hot encoding

X_val=x_train[20000:30000]
Y_val=to_categorical(y_train[20000:30000],10)

X_test=x_test
Y_test=to_categorical(y_test,10)

del x_train, y_train, x_test, y_test


print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
labels=np.array(["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"])
#sample image of each label
plt.figure(figsize=(15,15))
for i in range(0,len(np.unique(np.argmax(Y_train,axis=1)))):
    rmd=np.random.choice(np.where(np.argmax(Y_train,axis=1)==i)[0],1)
    plt.subplot(1,10,i+1)
    img=X_train[rmd]
    plt.imshow(img[0,:,:,:])
    plt.title(labels[i]+" "+str(np.argmax(Y_train,axis=1)[rmd][0]))

In [ ]:
# check the shape of the data
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

### RF on pixelvalues
In this section you will train a random forest on the raw pixelvalues of the images.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=40,random_state=22)
clf.fit(X_train.reshape(len(X_train),32*32*3), np.argmax(Y_train,axis=1))

In [ ]:
pred=clf.predict(X_test.reshape(len(X_test),32*32*3))
acc=np.average(pred==np.argmax(Y_test,axis=1))
res1 = pd.DataFrame(
          {'Acc' : acc}, index=['rf on pixelvalues'])
res1

### CNN from scratch without normalization
In this section you train a cnn from scratch to learn to classify the images into the right label. Normalization is not applied to the data.

In [ ]:
model  =  Sequential()

model.add(Convolution2D(16,(3,3),activation="relu",padding="same",input_shape=(32,32,3)))
model.add(Convolution2D(16,(3,3),activation="relu",padding="same"))
model.add(MaxPooling2D((2,2)))

model.add(Convolution2D(32,(3,3),activation="relu",padding="same"))
model.add(Convolution2D(32,(3,3),activation="relu",padding="same"))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:

# Convert one-hot labels to class indices
Y_train_indices = np.argmax(Y_train, axis=1)
Y_val_indices = np.argmax(Y_val, axis=1)
Y_test_indices = np.argmax(Y_test, axis=1)

# Create datasets
train_dataset = NumpyDataset(X_train, Y_train_indices, transform=None)
val_dataset = NumpyDataset(X_val, Y_val_indices, transform=None)
test_dataset= NumpyDataset(X_test, Y_test_indices, transform=None)

# From the datasets we create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

images, labels = next(iter(train_dataloader))
print(f'minimum value {images.min()} | maximum value {images.max()}')

In [ ]:
history =model.fit(train_dataloader, epochs=10, validation_data=val_dataloader)

In [ ]:
plot_history(history)

In [ ]:
acc=np.average(np.argmax(model.predict(test_dataloader),axis=1)==np.argmax(Y_test,axis=1))
res2 = pd.DataFrame(
          {'Acc' : acc}, index=['cnn from scratch without normalization']
)
pd.concat([res1,res2])

### CNN from scratch with normalization
In this section you train a cnn from scratch to learn to classify the images into the right label. Normalization is applied to the data.

In [ ]:
model  =  Sequential()

model.add(Convolution2D(16,(3,3),activation="relu",padding="same",input_shape=(32,32,3)))
model.add(Convolution2D(16,(3,3),activation="relu",padding="same"))
model.add(MaxPooling2D((2,2)))

model.add(Convolution2D(32,(3,3),activation="relu",padding="same"))
model.add(Convolution2D(32,(3,3),activation="relu",padding="same"))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
#  these transformatins make sure that the data leis in 0....1 range
train_transforms = tr.Compose([
    tr.ToPILImage(),               #  Convert numpy array to PIL image
    tr.ToTensor(),                 # Convert PIL image to Tensor
    tr.Lambda(lambda x: x.permute(1, 2, 0))  # Convert (C, H, W) -> (H, W, C)
])

val_test_transforms = tr.Compose([
    tr.ToPILImage(),               # Converting numpy array to PIL image
    tr.ToTensor(),                 # Convert PIL image to Tensor
    tr.Lambda(lambda x: x.permute(1, 2, 0))  # Convert (C, H, W) -> (H, W, C)
])


# Convert one-hot labels to class indices
Y_train_indices = np.argmax(Y_train, axis=1)
Y_val_indices = np.argmax(Y_val, axis=1)
Y_test_indices = np.argmax(Y_test, axis=1)
# Create datasets
train_dataset = NumpyDataset(X_train, Y_train_indices, transform=train_transforms)
val_dataset = NumpyDataset(X_val, Y_val_indices, transform=val_test_transforms)
test_dataset= NumpyDataset(X_test, Y_test_indices, transform=val_test_transforms)
# From the datasets we create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

images, labels = next(iter(train_dataloader))
print(f'minimum value {images.min()} | maximum value {images.max()}')


In [ ]:
history =model.fit(train_dataloader, epochs=10, validation_data=val_dataloader)

In [ ]:
plot_history(history)

In [ ]:
acc=np.average(np.argmax(model.predict(test_dataloader),axis=1)==np.argmax(Y_test,axis=1))
res3 = pd.DataFrame(
          {'Acc' : acc}, index=['cnn from scratch with normalization']
)
pd.concat([res1,res2,res3])

### CNN from scratch with Dropout
In this section you train a cnn from scratch to learn to classify the images into the right label. This time you will use dropout layers in the classification part. Normalization is not used.

In [ ]:
model  =  Sequential()

model.add(Convolution2D(16,(3,3),activation="relu",padding="same",input_shape=(32,32,3)))
model.add(Convolution2D(16,(3,3),activation="relu",padding="same"))
model.add(MaxPooling2D((2,2)))

model.add(Convolution2D(32,(3,3),activation="relu",padding="same"))
model.add(Convolution2D(32,(3,3),activation="relu",padding="same"))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dropout(0.3))   #<------ Droput layers
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.3))   #<------ Droput layers
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dropout(0.3))   #<------ Droput layers
model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Convert one-hot labels to class indices
Y_train_indices = np.argmax(Y_train, axis=1)
Y_val_indices = np.argmax(Y_val, axis=1)
Y_test_indices = np.argmax(Y_test, axis=1)

# Create datasets
train_dataset = NumpyDataset(X_train, Y_train_indices, transform=None)
val_dataset = NumpyDataset(X_val, Y_val_indices, transform=None)
test_dataset= NumpyDataset(X_test, Y_test_indices, transform=None)

# From the datasets we create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

images, labels = next(iter(train_dataloader))
print(f'minimum value {images.min()} | maximum value {images.max()}')

In [ ]:
history =model.fit(train_dataloader, epochs=15, validation_data=val_dataloader)

In [ ]:
plot_history(history)

In [ ]:
acc = np.average(np.argmax(model.predict(test_dataloader),axis=1)==np.argmax(Y_test,axis=1))
res4 = pd.DataFrame(
          {'Acc' : acc}, index=['cnn from scratch with dropout']
)
pd.concat([res1,res2,res3,res4])

### CNN from scratch with Batchnorm
In this section you train a cnn from scratch to learn to classify the images into the right label. This time you will use batchnorm on the input and in the convolutional part of the network. Note that we use the original images and do not normalize them.

In [ ]:
model  =  Sequential()

model.add(BatchNormalization(input_shape=(32,32,3)))
model.add(Convolution2D(16,(3,3),padding="same"))
model.add(BatchNormalization())                    #<------ Batchnorm layers
model.add(Activation('relu'))
model.add(Convolution2D(16,(3,3),padding="same"))
model.add(BatchNormalization())                    #<------ Batchnorm layers
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Convolution2D(32,(3,3),padding="same"))
model.add(BatchNormalization())                    #<------ Batchnorm layers
model.add(Activation('relu'))
model.add(Convolution2D(32,(3,3),padding="same"))
model.add(BatchNormalization())                    #<------ Batchnorm layers
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Convert one-hot labels to class indices
Y_train_indices = np.argmax(Y_train, axis=1)
Y_val_indices = np.argmax(Y_val, axis=1)
Y_test_indices = np.argmax(Y_test, axis=1)

# Create datasets
train_dataset = NumpyDataset(X_train, Y_train_indices, transform=None)
val_dataset = NumpyDataset(X_val, Y_val_indices, transform=None)
test_dataset= NumpyDataset(X_test, Y_test_indices, transform=None)

# From the datasets we create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

images, labels = next(iter(train_dataloader))
print(f'minimum value {images.min()} | maximum value {images.max()}')

In [ ]:
history =model.fit(train_dataloader, epochs=15, validation_data=val_dataloader)

In [ ]:
plot_history(history)

In [ ]:
acc = np.average(np.argmax(model.predict(test_dataloader),axis=1)==np.argmax(Y_test,axis=1))
res5 = pd.DataFrame(
          {'Acc' : acc}, index=['cnn from scratch with batchnorm']
)
pd.concat([res1,res2,res3,res4,res5])

#### Exercise
Calculate the confusion matrix of the networks.  
Play around with the dropout rate and the position of the batchnorm.

### CNN from scratch with Data Augmentation
In this section you train a cnn from scratch to learn to classify the images into the right label. This time you will use data augmentation, so the network will train on slightly different versions of the images in each epoch.

Data Augmentation is especially helpful if you do not have lots of data. Another approach to train with few data is to use a pretrained neural network. This will be covered in the next notebook.


In [ ]:
model  =  Sequential()

model.add(Convolution2D(16,(3,3),activation="relu",padding="same",input_shape=(32,32,3)))
model.add(Convolution2D(16,(3,3),activation="relu",padding="same"))
model.add(MaxPooling2D((2,2)))

model.add(Convolution2D(32,(3,3),activation="relu",padding="same"))
model.add(Convolution2D(32,(3,3),activation="relu",padding="same"))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=["accuracy"]
)

model.summary()

Note that for the augmentation we now need Dataloaders from torch.
It creates a dataset which can be loaded during traing , with the transformations we can specify what kind of augmentation each image should recieve.

In [ ]:
# Transformations pipeline for training
train_transforms = tr.Compose([
    tr.ToPILImage(),               #  Convert numpy array to PIL image
    tr.ToTensor(),                 # Convert PIL image to Tensor
    tr.RandomHorizontalFlip(),                                                           # Random horizontal flip
    tr.RandomRotation(15),                                                              # Random rotation
    tr.RandomCrop(32, padding=4),                                                       # Random cropping with padding
    tr.Lambda(lambda x: (x * 255.0).clamp(0, 255).permute(1, 2, 0))                     # Scale back to [0, 255] and return to (H, W, C)
])

# Convert one-hot labels to class indices
Y_train_indices = np.argmax(Y_train, axis=1)
Y_val_indices = np.argmax(Y_val, axis=1)
Y_test_indices = np.argmax(Y_test, axis=1)
# Create datasets
train_dataset = NumpyDataset(X_train, Y_train_indices, transform=train_transforms)
val_dataset = NumpyDataset(X_val, Y_val_indices, transform=None)
test_dataset= NumpyDataset(X_test, Y_test_indices, transform=None)
# From the datasets we create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



- see that the image is now randomly shifted cropped and flipped, each time the images enters ther training loop, this operation is happening on the fly while training the model


In [ ]:
images, labels = next(iter(train_dataloader))
print(images.min(), images.max())
plt.figure(figsize=(2,2))
plt.imshow(images[0].numpy().astype(np.uint8))
plt.show()

In [ ]:
history =model.fit(train_dataloader, epochs=15, validation_data=val_dataloader)


In [ ]:
plot_history(history)

In [ ]:
acc = np.average(np.argmax(model.predict(test_dataloader),axis=1)==np.argmax(Y_test,axis=1))
res6 = pd.DataFrame(
          {'Acc' : acc}, index=['cnn from scratch with data augmentation']
)
pd.concat([res1,res2,res3,res4,res5,res6])

## 🔧 **YOUR TASK:**
- Try to beat the performace of the best network with your own neural network.  
- you might want to combine some approaches from above
- dont forget to normalize also the testset if you use normalization (which you should use anyway😉)







<details>
  <summary>💡 Click here for a hint:</summary>



these transformatins normalize your data to 0- 1 and use augmentation

```
train_transforms = tr.Compose([
    tr.ToPILImage(),               #  Convert numpy array to PIL image
    tr.ToTensor(),                 # Convert PIL image to Tensor
    tr.RandomHorizontalFlip(),                                                           # Random horizontal flip
    tr.RandomRotation(15),                                                              # Random rotation
    tr.RandomCrop(32, padding=4),                                                       # Random cropping with padding
    tr.Lambda(lambda x: x.permute(1, 2, 0))                     # Scale back to [0, 255] and return to (H, W, C)
])
val_test_transforms = tr.Compose([
    tr.ToPILImage(),               # Converting numpy array to PIL image
    tr.ToTensor(),                 # Convert PIL image to Tensor
    tr.Lambda(lambda x: x.permute(1, 2, 0))
  ])
# Convert one-hot labels to class indices
Y_train_indices = np.argmax(Y_train, axis=1)
Y_val_indices = np.argmax(Y_val, axis=1)
Y_test_indices = np.argmax(Y_test, axis=1)
# Create datasets
train_dataset = NumpyDataset(X_train, Y_train_indices, transform=train_transforms)
val_dataset = NumpyDataset(X_val, Y_val_indices, transform=val_test_transforms)
test_dataset= NumpyDataset(X_test, Y_test_indices, transform=val_test_transforms)
# From the datasets we create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
```


</details>

In [ ]:
### YOUR CODE ###